# Jupyter Notebook for the evaluation of nucleon PDFs from Distillation correlation functions

## Initialization

In [ ]:
import numpy as np
import scipy.linalg as la
import scipy.optimize as scipyOpt
import math, itertools as it
import matplotlib as mpl ; import matplotlib.pyplot as plt ; plt.ion()
from matplotlib import rc
import pickle
#rc('font',**{'family':'sans-serif','sans-serif':['Helvetiva']})
rc('text', usetex=True)

print ("Done")

## Directories, Parameters, Constants

### Generic definitions

In [ ]:
def dispTag(d):
    dL = lambda i: ("z" if i != 0 else "") + ("+" if i > 0 else "") + str(i)
    return 'disp_%s'%(dL(d))

def MomTag(zMom):
    mSgn = lambda i: ("+" if i > 0 else "") + str(i)
    return 'pz.%s'%(mSgn(zMom))

def phMomTpl(phTag,zMom):
    return (phTag,MomTag(zMom))

def momFileTag(mVec):
    mFTag = 'momXYZ.%d.%d.%d' % (mVec[0],mVec[1],mVec[2])
    return mFTag
#-----------------------------------

phaseTypes    = ['unphased']
SrcSnkOpTypes = ['single']
maxDisp = 8
dispLen = np.arange(-maxDisp,maxDisp+1)


# Parameters for unphased correlators with a single src-sink operator
phTS = 'unphased'
opTS = 'single'

# Note: Every combination of momentum, t0 and t_sink is considered a 'dataset'
ZmomVal   = {(phTS,opTS): [0,1,2]}#,3,-1,-2,-3]} # Available momenta values
TsinkList = {(phTS,opTS): [4,6,8,10,12,14]}   # Tsink Values
T0List    = {(phTS,opTS): [0,16,32,48]}       # T0 Values

# Insertion gamma matrices
gMatList = ['gt',
            'gxg5','gyg5','gzg5','gtg5',
            'gxgy','gxgz','gxgt','gygz','gygt','gzgt']

# (Maximum) Number of configs
Ncfg_all  = 349

# Base Directory of two- and three-point correlation functions
corr2ptWorkDir = '../2pt_corr_effEnergy'
corr3ptMainDir = {(phTS,opTS): '3pt_data_%s'%(phTS)}
corr2ptMainDir = {(phTS,opTS): '%s/2pt_data_%s'%(corr2ptWorkDir,phTS)}

momTags_Dict = {(phTS,opTS): []}
momVector_Dict  = {}
Ncfg_Dict_3pt = {}
Ncfg_Dict_2pt = {}
existsDataSet_3pt = {}
existsDataSet_2pt = {}
Nt_Dict_2pt = {}
for zmom in ZmomVal[(phTS,opTS)]:
    mTag = MomTag(zmom)
    momTags_Dict[(phTS,opTS)].append(mTag)
    momVector_Dict[mTag] = np.array([0,0,zmom])
    
    for tsnk in TsinkList[(phTS,opTS)]:
        dTpl = (phTS,opTS,mTag,tsnk)
        Ncfg_Dict_3pt[dTpl] = Ncfg_all  # All datasets have the same number of configs
        for t0 in T0List[(phTS,opTS)]:
            dTpl3pt = (phTS,opTS,mTag,tsnk,t0)
            existsDataSet_3pt[dTpl3pt] = True  # Whether all possible datasets exist for the three-point function

    dTpl2 = (phTS,opTS,mTag)
    Ncfg_Dict_2pt[dTpl2] = Ncfg_all            
    for t0 in T0List[(phTS,opTS)]:
        dTpl2pt = (phTS,opTS,mTag,t0)
        existsDataSet_2pt[dTpl2pt] = True  # Whether all possible datasets exist for the two-point function
        
    Nt_Dict_2pt[phMomTpl(phTS,zmom)] = 21 # 2pt for all momenta have 21 time-slices        
#----------------------------------------------


# Some data for the two-point function do not exist for now, skip those
# for zmom in [3,-1,-2,-3]:
#     mTag = MomTag(zmom)
#     for t0 in T0List[(phTS,opTS)]:
#         dTpl2pt = (phTS,opTS,mTag,t0)
#         existsDataSet_2pt[dTpl2pt] = False

# Zero-momentum for t0=48 does not exist, skip it
zmom = 0
t0 = 48
mTag = MomTag(zmom)
dTpl2pt = (phTS,opTS,mTag,t0)
existsDataSet_2pt[dTpl2pt] = False
#----------------------------------------------


# Which datasets to analyze (if there's at least one t0 for each momentum and tsink that's OK)
analyzeDataSetM = {}
for mTag in momTags_Dict[(phTS,opTS)]:
    for tsnk in TsinkList[(phTS,opTS)]:
        adTplM = (phTS,opTS,mTag,tsnk)
        for t0 in T0List[(phTS,opTS)]:
            dTpl3pt = (phTS,opTS,mTag,tsnk,t0)
            dTpl2pt = (phTS,opTS,mTag,t0)
            if existsDataSet_3pt[dTpl3pt] and existsDataSet_2pt[dTpl2pt]:
                analyzeDataSetM[adTplM] = True
            else:
                analyzeDataSetM[adTplM] = False
            break

analyzeDataSetE = {}
for mTag in momTags_Dict[(phTS,opTS)]:
    adTplE = (phTS,opTS,mTag)
    for t0 in T0List[(phTS,opTS)]:
        dTpl2pt = (phTS,opTS,mTag,t0)
        if existsDataSet_2pt[dTpl2pt]:
            analyzeDataSetE[adTplE] = True
        else:
            analyzeDataSetE[adTplE] = False
        break        
#----------------------------------------------


print('Momentum Tags:\n', momTags_Dict)
print('\n')
print('Momentum Vectors:\n', momVector_Dict)

print('\nDone\n')

### Read/define source-insertion-sink operators

In [ ]:

# Interpolating operators  for 3-point functions         
nRow3pt_Dict = {'single': {}}
nRow2pt_Dict = {'single': {}}
srcOpList3pt_Dict = {'single': {}}
snkOpList3pt_Dict = {'single': {}}
srcOpList2pt_Dict = {'single': {}}
snkOpList2pt_Dict = {'single': {}}
for zmom in ZmomVal[(phTS,opTS)]:
    pmTpl = phMomTpl(phTS,zmom)

    # 2-point function interpolators have one row for zmom = 0
    if zmom == 0:
        nRow2pt_Dict['single'][pmTpl] = 1
    else:
        nRow2pt_Dict['single'][pmTpl] = 2
    
    # 3-point function interpolators have two-rows for all momenta
    nRow3pt_Dict['single'][pmTpl] = 2

    if zmom == 0:
        srcOpList3pt_Dict['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_G1g1']
        snkOpList3pt_Dict['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_G1g1']
        srcOpList2pt_Dict['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_G1g1']
        snkOpList2pt_Dict['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_G1g1']
    else:
        srcOpList3pt_Dict['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_H1o2D4E1']
        snkOpList3pt_Dict['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_H1o2D4E1']
        srcOpList2pt_Dict['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_H1o2D4E1']
        snkOpList2pt_Dict['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_H1o2D4E1']
#----------------------------------------------


# Insertion operators list, that's a common file among all momenta
insOpFile_Dict = {phTS: '../pPDF/insert_gamma_row.DA.list'}

# Read insertion operators
insOptrList = {}
insOpFname = insOpFile_Dict[phTS]
insOptrList[phTS] = {'name': [], 'row': []}
with open(insOpFname) as fp:
    line = fp.readlines()
    for t in line:
        insOptrList[phTS]['name'].append(t.split()[0])
        insOptrList[phTS]['row'].append(int(t.split()[1]))

# Create dictionary to map actual gamma matrices to operator names and rows
gMatMap = {phTS: {'gt'  : ('b_b0xDA__J0_A1pP', 1),
                  'gxg5': ('a_a1xDA__J1_T1pM', 1),
                  'gyg5': ('a_a1xDA__J1_T1pM', 3),
                  'gzg5': ('a_a1xDA__J1_T1pM', 2),
                  'gtg5': ('pion_pion_2xDA__J0_A1mM', 1),
                  'gxgy': ('b_b1xDA__J1_T1pP', 2),
                  'gxgz': ('b_b1xDA__J1_T1pP', 3),
                  'gxgt': ('rho_rho_2xDA__J1_T1mP', 3),
                  'gygz': ('b_b1xDA__J1_T1pP', 1),
                  'gygt': ('rho_rho_2xDA__J1_T1mP', 1),
                  'gzgt': ('rho_rho_2xDA__J1_T1mP', 2)}}
#----------------------------------------------


# Create dictionary to map actual gamma matrices to latex format
gMatLatex = {'gt'  : '\gamma_t',
             'gxg5': '\gamma_x\gamma_5',
             'gyg5': '\gamma_y\gamma_5',
             'gzg5': '\gamma_z\gamma_5',
             'gtg5': '\gamma_t\gamma_5',
             'gxgy': '\gamma_x\gamma_y',
             'gxgz': '\gamma_x\gamma_z',
             'gxgt': '\gamma_x\gamma_t',
             'gygz': '\gamma_y\gamma_z',
             'gygt': '\gamma_y\gamma_t',
             'gzgt': '\gamma_z\gamma_t'}
#----------------------------------------------



print('Insertion operator List:\n', insOptrList[phTS])

print('\nDone\n')

### Ensemble-related definitions

In [ ]:
# Ensemble parameters
L = 32
T = 64

binsize=1
#------------------------------------------------

# Nucleon, pion mass
alat_fm = 0.094
alat_iGeV = alat_fm / 0.1973
mpi_GeV = 0.350
mN_GeV = 1.123
amN = mN_GeV * alat_iGeV

# Dispersion relation
Pi = np.pi
def dispRel(am,k):
    ap = 2*Pi/L * k
    return np.sqrt(am**2 + np.dot(ap,ap))

print('Done\n')

### Plotting dictionaries

In [ ]:
pDct   = {0: 'ro', 1: 'bs', 2: 'g^', 3: 'mp', 4: 'cX'}
cDct   = {0: 'r', 1: 'b', 2: 'g', 3: 'm', 4: 'c', 5: 'darkorange', 6: 'darkviolet', 7: 'gold'}
mDct   = {0: 'o', 1: 's', 2: '^', 3: 'p', 4: 'X', 5: 'D', 6: 'v', 7: '>'}
osDct  = {0: 0.0 , 1: 0.15, 2: 0.3, 3: 0.45, 4: 0.6}
osDctF = {0: 0.0 , 1: 0.07, 2: 0.15, 3: 0.22, 4: 0.3,
          5: 0.37, 6: 0.45, 7: 0.52, 8: 0.60, 9: 0.67}
fsDct = {1: 'full',2:'none'}

### Jackknife-related functions

In [ ]:
def jackknife_ave(bins,Nbins,tL):

    Jax = 0
    
    if(tL==1):
        ave = np.mean(bins,dtype=bins.dtype)

        sqsum = sum(map(lambda x:x*x,ave-bins))
        fac = (Nbins -1) / float(Nbins)
        err = np.sqrt(fac*sqsum)
    else:
        ave = np.mean(bins, axis=Jax,dtype=bins.dtype)

        err = np.zeros(tL)
        for i in range(tL):
#            sqsum = sum(map(lambda x:x*x,ave[i]-bins[:,i].real))
            sqsum = sum(map(lambda x:x*x,ave[i]-bins[:,i]))
            fac = (Nbins -1) / float(Nbins)
            err[i] = np.sqrt(fac*sqsum.real)        
            
    return (ave,err)
#-------------------------------------------------------

def jackknife_binning(corr, binsize=1):
    
    Jax=0
    
    # Define the number of bins
    mod   = Ncfg%binsize
    Nbins = int((Ncfg - mod) / binsize)
    bins = np.zeros((Nbins,Nt),dtype=corr.dtype)

    csum = np.sum(corr,axis=Jax) # Sum w.r.t to the configurations
    for t in range(Nt):
        for m in np.arange(1,mod+1):
            csum[t] -= corr[Ncfg-m,t]  # Throw away data in case there is modulo
            
        for b in range(Nbins):
            bsum = 0
            for k in range(binsize):
                bsum += corr[b*binsize+k,t]
                
            bins[b][t] = (csum[t] - bsum) / float(Ncfg - binsize - mod) # Bin averages for each bin,t

    return bins, Nbins
#-------------------------------------------------------

print('Done\n')

### Fit functions

In [ ]:
# Constant fit
def constFit(x,E0):
    return E0
#------------------------

print('Done\n')

## Read two-point functions, perform Jackknife sampling

In [ ]:
def get2ptFileName(phT,t0Tag,src,snk,srow,mFTag):
    filePre = 'corr_2pt.baryon.n64'
    
    srcTag = 'src_%s_%d'%(src,srow)
    snkTag = 'snk_%s_%d'%(snk,srow)
    
    FileName = '%s.%s.%s.%s.%s.%s.dat'%(filePre,phT,t0Tag,srcTag,snkTag,mFTag)
    
    return FileName
#--------------------------

def get2ptFileDir(mainDir,t0Tag,mFTag):
    return '%s/%s/%s' % (mainDir,t0Tag,mFTag)
#--------------------------------------------------------------


c2pt_Allbins   = {}
c2pt_Allave    = {}
c2pt_rowAvg    = {}
c2pt_rawAllave = {}
c2pt_bins      = {}
c2pt_ave       = {}
c2pt_rawAve    = {}

Nbins_2pt_Dict = {}

effEnergy_Allbins = {}
effEnergy_Allave  = {}
effEnergy_bins    = {}
effEnergy_ave     = {}


for opT in SrcSnkOpTypes:
    for phT in phaseTypes:
        momTags = momTags_Dict[(phT,opT)]
                
        for mTag in momTags:
            mVec = momVector_Dict[mTag]
            zMom = mVec[2]
            mFTag = momFileTag(mVec)
            pmTpl = phMomTpl(phT,zMom)

            nRow = nRow2pt_Dict[opT][pmTpl]

            srcOpList2pt = srcOpList2pt_Dict[opT][pmTpl]        
            snkOpList2pt = srcOpList2pt_Dict[opT][pmTpl]

            dTpl = (phT,opT,mTag)
            Ncfg = Ncfg_Dict_2pt[dTpl]

            Nt = Nt_Dict_2pt[pmTpl]

            # Determine number of t0 we consider
            Nt0 = 0
            for t0 in T0List[(phT,opT)]:
                dTpl2pt = (phT,opT,mTag,t0)
                if existsDataSet_2pt[dTpl2pt]:
                    Nt0 += 1

            for isrc, src in enumerate(srcOpList2pt):
                for isnk, snk in enumerate(snkOpList2pt):
                    ckey = (opT,phT,mTag,isrc,isnk) # All info in one tuple

                    c2pt_rawAll = np.zeros((Ncfg,Nt,nRow,Nt0),dtype=np.complex128)

                    for it0,t0 in enumerate(T0List[(phT,opT)]):
                        dTpl2pt = (phTS,opTS,mTag,t0)
                        if existsDataSet_2pt[dTpl2pt]:
                            t0Tag = 't0_%d'%(t0)

                            for ir,irow in enumerate(range(1,nRow+1)):
                                ckeyA = (opT,phT,mTag,isrc,isnk,t0,irow) # All info in one tuple

                                c2ptFileDir  = get2ptFileDir(corr2ptMainDir[(phT,opT)],t0Tag,mFTag)
                                c2ptFileName = get2ptFileName(phT,t0Tag,src,snk,irow,mFTag)               
                                c2ptFile = '%s/%s' % (c2ptFileDir,c2ptFileName)

                                with open(c2ptFile) as fp:
                                    line = fp.readlines()
                                    c = 0
                                    for n in line:
                                        it   = c%Nt
                                        icfg = c//Nt
                                        c2pt_rawAll[icfg,it,ir,it0] = complex(np.float64(n.split()[1]),
                                                                              np.float64(n.split()[2]))
                                        c += 1
                                # Reading file

                                # Jackknife average on All two-point functions
                                c2pt_Allbins[ckeyA], Nbins_2pt_Dict[dTpl] = jackknife_binning(c2pt_rawAll[:,:,ir,it0].real)
                                Nbins_2pt = Nbins_2pt_Dict[dTpl]
                                c2pt_Allave[ckeyA] = jackknife_ave(c2pt_Allbins[ckeyA],Nbins_2pt,tL=Nt)

                                # Effective energy of All t0 and rows
                                effEnergy_Allbins[ckeyA] = np.zeros((Nbins_2pt,Nt))
                                for it in range(Nt):
                                    effEnergy_Allbins[ckeyA][:,it] = np.log(c2pt_Allbins[ckeyA][:,it]/
                                                                            c2pt_Allbins[ckeyA][:,(it+1)%Nt])

                                effEnergy_Allave[ckeyA] = jackknife_ave(effEnergy_Allbins[ckeyA],Nbins_2pt,Nt)                                
                            # for-irow

                            # Standard average and standard error (still average over rows)
                            # Might need this for constructing covariance matrix
                            ckeyB = (opT,phT,mTag,isrc,isnk,t0)
                            c2pt_rowAvg[ckeyB] = np.mean(c2pt_rawAll[:,:,:,it0],axis=2,dtype=np.complex128) # Average over rows
                            c2pt_rawAllave[ckeyB] = (np.mean(c2pt_rowAvg[ckeyB],axis=0),
                                                        np.std(c2pt_rowAvg[ckeyB],axis=0)/np.sqrt(Ncfg))
                        else:
                            print('### Skipping dataset:', dTpl2pt)
                    # for-t0

                    # Average over rows and t0
                    c2pt_Tmp1 = np.mean(c2pt_rawAll,axis=3,dtype=np.complex128) # over t0
                    c2pt_Tmp2 = np.mean(c2pt_Tmp1  ,axis=2,dtype=np.complex128) # over rows

                    # Standard average and standard error
                    c2pt_rawAve[ckey] = (np.mean(c2pt_Tmp2,axis=0),np.std(c2pt_Tmp2,axis=0)/np.sqrt(Ncfg))

                    # Jackknife-average on t0- and row-average            
                    c2pt_bins[ckey], Nbins_2pt = jackknife_binning(c2pt_Tmp2.real)
                    c2pt_ave[ckey] = jackknife_ave(c2pt_bins[ckey],Nbins_2pt,tL=Nt)

                    effEnergy_bins[ckey] = np.zeros((Nbins_2pt,Nt))
                    for it in range(Nt):
                        effEnergy_bins[ckey][:,it] = np.log(c2pt_bins[ckey][:,it]/
                                                            c2pt_bins[ckey][:,(it+1)%Nt])

                    effEnergy_ave[ckey] = jackknife_ave(effEnergy_bins[ckey],Nbins_2pt,tL=Nt)                    
                    
            # for-src-snk
            print('Done: Momentum %s'%(mTag))
        # for momentum
        
           
print('\nDone\n')

## Perform constant fit on the Effective Energy

In [ ]:

tini = 7
tfin = 13
tFit = np.arange(tini,tfin+1)
Ndata = len(tFit)

E0_fit = {}
E0_cov = {}

E0_fit_ave = {}
for opT in SrcSnkOpTypes:
    for phT in phaseTypes:
        momTags = momTags_Dict[(phT,opT)]
                
        for mTag in momTags:
            adTplE = (phT,opT,mTag)
            if analyzeDataSetE[adTplE]:
                mVec = momVector_Dict[mTag]
                zMom = mVec[2]
                pmTpl = phMomTpl(phT,zMom)

                srcOpList2pt = srcOpList2pt_Dict[opT][pmTpl]        
                snkOpList2pt = srcOpList2pt_Dict[opT][pmTpl]

                dTpl = (phT,opT,mTag)
                Nbins = Nbins_2pt_Dict[dTpl]

                for isrc, src in enumerate(srcOpList2pt):
                    for isnk, snk in enumerate(snkOpList2pt):
                        ckey = (opT,phT,mTag,isrc,isnk) # All info in one tuple

                        E0_fit[ckey] = np.zeros(Nbins)
                        E0_cov[ckey] = np.zeros((Nbins,Ndata))

                        E0_fitAll = np.zeros(Nbins)
                        E0_covAll = np.zeros((Nbins,Ndata))

                        nFit = 0
                        for ib in range(Nbins):
                            ydata = effEnergy_bins[ckey][ib,tini:tfin+1]
                            yerr  = effEnergy_ave[ckey][1][tini:tfin+1] 
                            if (not np.isnan(ydata).any()):
                                E0_fitAll[nFit], E0_covAll[nFit] = scipyOpt.curve_fit(constFit,tFit,ydata,sigma=yerr)
                                nFit += 1

                        E0_fit[ckey] =  E0_fitAll[:nFit]

                        if (np.shape(E0_fit[ckey])==(0,)) or (np.shape(E0_fit[ckey])==(1,)):
                            E0_fit_ave[ckey] = (0,0)
                        else:
                            E0_fit_ave[ckey] = jackknife_ave(E0_fit[ckey],Nbins=nFit,tL=1)
            else:
                print('Skipping dataset: ',adTplE)

            print('%s Done'%(mTag))
        # for-momentum
            
    
print('\nDone\n')


## Plot Effective energy

In [ ]:
# Two-point function
fig1, ax1 = plt.subplots(figsize=[15,9.4])
fig2, ax2 = plt.subplots(figsize=[15,9.4])
#fig3, ax3 = plt.subplots(figsize=[15,9.4])

elw=1
ecs=6

iSrc = 0 #srcOpList.index('NucleonMG1g1MxD0J0S_J1o2_H1o2D4E1')
iSnk = 0 #snkOpList.index('NucleonMG1g1MxD0J0S_J1o2_H1o2D4E1')
print('isrc: %s\nisnk: %s'%(iSrc,iSnk))

phT = None
opT = None
phTP = 'unphased'
opTP = 'single'

# Plot one momentum, all effective energies
mTag = 'pz.+1'
mVec = momVector_Dict[mTag]
zMom = mVec[2]
pmTpl = phMomTpl(phTP,zMom)

Nt = Nt_Dict_2pt[pmTpl]

runTag = None#'%s_%s'%(phaseInfo,pL)

nRow = nRow2pt_Dict[opTP][pmTpl]
xRng = np.arange(Nt)

i=0
osi=0
for t0 in T0List[(phTP,opTP)]:
    dTpl2pt = (phTP,opTP,mTag,t0)
    if existsDataSet_2pt[dTpl2pt]:

        for irow in range(1,nRow+1):
            ckey = (opTP,phTP,mTag,iSrc,iSnk,t0,irow)

            ax1.errorbar(xRng+osDctF[osi],  effEnergy_Allave[ckey][0].real,
                         yerr = effEnergy_Allave[ckey][1],
                         fmt = pDct[i], label=r'$t_0=%d$, row=%d'%(t0,irow),
                         ms = 8.0, elinewidth=elw, capsize=ecs,fillstyle=fsDct[irow])

            ax1.axhline(dispRel(amN,mVec),lw=1,color='k',ls='--')

            osi+=1
        # for-irow
        i+=1
    else:
        print('Skipping ', dTpl2pt)

ckey = (opTP,phTP,mTag,iSrc,iSnk)
ax1.errorbar(xRng-0.2,  effEnergy_ave[ckey][0].real,
             yerr = effEnergy_ave[ckey][1],
             fmt = 'kX', label=r'$\vec{p}=(%d,%d,%d)$ Avg.'%(mVec[0],mVec[1],mVec[2]),
             ms = 9.0, elinewidth=2, capsize=ecs)#,fillstyle='none')
    
ax1.text(0,0.6,r'$\vec{p}=(%d,%d,%d)$'%(mVec[0],mVec[1],mVec[2]),fontsize=18)    

ax1.set_xlabel(r'$t/a$',fontsize=18)
ax1.set_ylabel(r'$aE_{\rm eff}(t)$',fontsize=18)
ax1.xaxis.set_ticks(np.arange(0, 20, 2))

ax1.set_ylim(0.5,1.2)
ax1.yaxis.set_ticks(np.arange(0.45, 0.86, 0.05))

ax1.legend(loc='best', ncol=2, numpoints=1, fontsize=18,
           labelspacing=0.2,borderpad=0.2,columnspacing=0.2,
           markerscale=0.8)

fig1.savefig('%s/aEeff.%s.all_t0.rowAll.pdf'%(corr2ptWorkDir,momFileTag(mVec)), bbox_inches='tight')

#-------------------------------------------------------------------------------------

# Plot average Eff. energy for all momenta
i=0
for mTag in momTags_Dict[(phTP,opTP)]:
    mVec = momVector_Dict[mTag]
    zMom = mVec[2]
    adTplE = (phTP,opTP,mTag)
    if analyzeDataSetE[adTplE] and (np.abs(zMom)<3):
        mVec = momVector_Dict[mTag]
        zMom = mVec[2]
        pmTpl = phMomTpl(phTP,zMom)

        Nt = Nt_Dict_2pt[pmTpl]

        runTag = None#'%s_%s'%(phaseInfo,pL)
        nRow = nRow2pt_Dict[opTP][pmTpl]
        xRng = np.arange(Nt)

        ckey = (opTP,phTP,mTag,iSrc,iSnk)

        ax2.errorbar(xRng,  effEnergy_ave[ckey][0].real,
                     yerr = effEnergy_ave[ckey][1],
                     fmt = pDct[i], label=r'$\vec{p}=(%d,%d,%d)$'%(mVec[0],mVec[1],mVec[2]),
                     ms = 8.0, elinewidth=elw, capsize=ecs)

        ax2.axhline(dispRel(amN,mVec),lw=1,color=cDct[i],ls='--')

        cFitv = E0_fit_ave[ckey][0]
        cFite = E0_fit_ave[ckey][1]
        if cFitv != 0 and cFite != 0:
            ax2.fill_between(tFit, cFitv-cFite, cFitv+cFite, facecolor=cDct[i],
                             alpha=0.4)#, label= 'const. fit')
        else:
            print('Fit for ', ckey, 'was not proper')

        i+=1
    
ax2.set_xlabel(r'$t/a$',fontsize=18)
ax2.set_ylabel(r'$aE_{\rm eff}(t)$',fontsize=18)
ax2.xaxis.set_ticks(np.arange(0, 22, 2))

ax2.set_ylim(0.45,0.85)
ax2.yaxis.set_ticks(np.arange(0.45, 0.86, 0.05))

ax2.legend(loc='best', ncol=1, numpoints=1, fontsize=18,
           labelspacing=0.2,borderpad=0.2,columnspacing=0.2,
           markerscale=0.8)

fig2.savefig('%s/aEeff.allMom.t0Ave.rowAve.pdf'%(corr2ptWorkDir),bbox_inches='tight')

#-------------------------------------------------------------------------------------


## Read three-point functions, perform Jackknife sampling

In [ ]:
def get3ptFileName(phT,t0Tag,tsnkTag,src,snk,srcRow,snkRow,ins,insRow,mFTag,d):
    filePre = 'corr_3pt.baryon.n64'
    
    srcTag = 'src_%s_%d'%(src,srcRow)
    snkTag = 'snk_%s_%d'%(snk,snkRow)
    insTag = 'ins_%s_%d'%(ins,insRow)
    
    dTag = dispTag(d)
    
    FileName = '%s.%s.%s.%s.%s.%s.%s.%s.%s.dat'%(filePre,phT,t0Tag,tsnkTag,srcTag,snkTag,insTag,dTag,mFTag)
    
    return FileName
#--------------------------

def get3ptFileDir(mainDir,t0Tag,mFTag,tsnkTag):
    return '%s/%s/%s/%s' % (mainDir,t0Tag,mFTag,tsnkTag)
#--------------------------------------------------------------


c3pt_Allbins_Re = {}
c3pt_Allbins_Im = {}
c3pt_Allave_Re  = {}
c3pt_Allave_Im  = {}

c3pt_rawAll = {}

for opT in SrcSnkOpTypes:
    for phT in phaseTypes:
        momTags = momTags_Dict[(phT,opT)]
                
        for mTag in momTags:
            mVec = momVector_Dict[mTag]
            zMom = mVec[2]
            mFTag = momFileTag(mVec)
            pmTpl = phMomTpl(phT,zMom)

            nRow = nRow3pt_Dict[opT][pmTpl]

            srcOpList3pt = srcOpList3pt_Dict[opT][pmTpl]        
            snkOpList3pt = srcOpList3pt_Dict[opT][pmTpl]        

            for tsnk in TsinkList[(phT,opT)]:
                tsnkTag = 'tsnk_%d'%(tsnk)
                dTpl = (phT,opT,mTag,tsnk)
                Ncfg = Ncfg_Dict_3pt[dTpl]
                Nt = tsnk
                
                for it0,t0 in enumerate(T0List[(phT,opT)]):
                    t0Tag = 't0_%d'%(t0)
                    
                    dTpl3pt = (phT,opT,mTag,tsnk,t0)
                    dTpl2pt = (phTS,opTS,mTag,t0)
                    if existsDataSet_3pt[dTpl3pt] and existsDataSet_2pt[dTpl2pt]:
                        print('Reading dataset: ', dTpl3pt)
                        for isrc, src in enumerate(srcOpList3pt):
                            for isnk, snk in enumerate(snkOpList3pt):
                                for gMat in gMatList:
                                    insOpName = gMatMap[phT][gMat][0]
                                    insOpRow  = gMatMap[phT][gMat][1]

                                    for irowSrc in range(1,nRow+1):
                                        for irowSnk in range(1,nRow+1):

                                            for d in dispLen:
                                                ckeyA = (opT,phT,mTag,tsnk,isrc,isnk,gMat,d,irowSrc,irowSnk,t0) # All info in one tuple
                                                c3pt_rawAll[ckeyA] = np.zeros((Ncfg,Nt),dtype=np.complex128)

                                                c3ptFileDir  = get3ptFileDir(corr3ptMainDir[(phT,opT)],t0Tag,mFTag,tsnkTag)
                                                c3ptFileName = get3ptFileName(phT,t0Tag,tsnkTag,src,snk,irowSrc,irowSnk,
                                                                              insOpName,insOpRow,mFTag,d)               
                                                c3ptFile = '%s/%s' % (c3ptFileDir,c3ptFileName)

                                                with open(c3ptFile) as fp:
                                                    line = fp.readlines()
                                                    c = 0
                                                    for n in line:
                                                        it   = c%Nt
                                                        icfg = c//Nt
                                                        c3pt_rawAll[ckeyA][icfg,it] = complex(np.float64(n.split()[1]),
                                                                                              np.float64(n.split()[2]))
                                                        c += 1
                                                # Reading file

                                                # Jackknife average on All three-point functions
                                                c3pt_Allbins_Re[ckeyA], Nbins_tmp1 = jackknife_binning(c3pt_rawAll[ckeyA].real)
                                                c3pt_Allbins_Im[ckeyA], Nbins_tmp2 = jackknife_binning(c3pt_rawAll[ckeyA].imag)
                                                c3pt_Allave_Re[ckeyA] = jackknife_ave(c3pt_Allbins_Re[ckeyA],Nbins_tmp1,tL=Nt)
                                                c3pt_Allave_Im[ckeyA] = jackknife_ave(c3pt_Allbins_Im[ckeyA],Nbins_tmp2,tL=Nt)
                                            # for-displacement
                                    # for-Rows
                        # for-src-snk-ins
                    else:
                        print('### Skipping dataset:', dTpl3pt)
                # for-t0
                
                print('Done: Momentum %s , t_sink = %d'%(mTag,tsnk))
            # for tsnk
        # for momentum
             
print('\nDone\n')

### Dump three-point function data into a Pickle file

In [ ]:

pickle.dump(c3pt_rawAll    , open( 'c3pt_rawAll.p'    , 'wb' ))
# pickle.dump(c3pt_Allbins_Re, open( 'c3pt_Allbins_Re.p', 'wb' ))
# pickle.dump(c3pt_Allbins_Im, open( 'c3pt_Allbins_Im.p', 'wb' ))
# pickle.dump(c3pt_Allave_Re , open( 'c3pt_Allave_Re.p' , 'wb' ))
# pickle.dump(c3pt_Allave_Im , open( 'c3pt_Allave_Im.p' , 'wb' ))

print('\nDone\n')

### Average three-point functions over $t_0$, perform Jackknife sampling

In [ ]:
c3pt_rawAve = {}

c3pt_bins_Re = {}
c3pt_bins_Im = {}

c3pt_ave_Re = {}
c3pt_ave_Im = {}

Nbins_3pt_Dict = {}

for opT in SrcSnkOpTypes:
    for phT in phaseTypes:
        momTags = momTags_Dict[(phT,opT)]
                
        for mTag in momTags:
            mVec = momVector_Dict[mTag]
            zMom = mVec[2]
            mFTag = momFileTag(mVec)
            pmTpl = phMomTpl(phT,zMom)

            nRow = nRow3pt_Dict[opT][pmTpl]

            srcOpList3pt = srcOpList3pt_Dict[opT][pmTpl]        
            snkOpList3pt = srcOpList3pt_Dict[opT][pmTpl]        

            for tsnk in TsinkList[(phT,opT)]:
                tsnkTag = 'tsnk_%d'%(tsnk)
                dTpl = (phT,opT,mTag,tsnk)
                Ncfg = Ncfg_Dict_3pt[dTpl]
                Nt = tsnk
                
                adTplM = (phT,opT,mTag,tsnk)
                if analyzeDataSetM[adTplM]:                
                    for isrc, src in enumerate(srcOpList3pt):
                        for isnk, snk in enumerate(snkOpList3pt):
                            for gMat in gMatList:
                                insOpName = gMatMap[phT][gMat][0]
                                insOpRow  = gMatMap[phT][gMat][1]

                                for irowSrc in range(1,nRow+1):
                                    for irowSnk in range(1,nRow+1):

                                        for d in dispLen:
                                            ckey = (opT,phT,mTag,tsnk,isrc,isnk,gMat,d,irowSrc,irowSnk) # All info in one tuple


                                            # Average over t0
                                            c3pt_t0_ave = np.zeros((Ncfg,Nt),dtype=np.complex128)
                                            Nt0 = 0
                                            for it0,t0 in enumerate(T0List[(phT,opT)]):
                                                ckeyA = (opT,phT,mTag,tsnk,isrc,isnk,gMat,d,irowSrc,irowSnk,t0) # All info in one tuple

                                                dTpl3pt = (phT,opT,mTag,tsnk,t0)
                                                dTpl2pt = (phTS,opTS,mTag,t0)
                                                if existsDataSet_3pt[dTpl3pt] and existsDataSet_2pt[dTpl2pt]:
                                                    Nt0 += 1
                                                    c3pt_t0_ave = c3pt_t0_ave + c3pt_rawAll[ckeyA]                                        
                                            # for-t0
                                            c3pt_t0_ave = c3pt_t0_ave / float(Nt0)


                                            # Standard average and standard error
                                            c3pt_rawAve[ckey] = (np.mean(c3pt_t0_ave,axis=0,dtype=np.complex128),
                                                                 np.std(c3pt_t0_ave,axis=0,dtype=np.complex128)/np.sqrt(Ncfg))

                                            # Jackknife-average on t0- and row-average            
                                            c3pt_bins_Re[ckey], Nbins_3pt_Dict[dTpl] = jackknife_binning(c3pt_t0_ave.real)
                                            c3pt_bins_Im[ckey], Nbins_3pt_tmp3       = jackknife_binning(c3pt_t0_ave.imag)

                                            c3pt_ave_Re[ckey] = jackknife_ave(c3pt_bins_Re[ckey],Nbins_3pt_Dict[dTpl],tL=Nt)
                                            c3pt_ave_Im[ckey] = jackknife_ave(c3pt_bins_Im[ckey],Nbins_3pt_Dict[dTpl],tL=Nt)

                                    # for-displacement
                    # for-src-snk-ins
                    print('Done: Momentum %s , t_sink = %d'%(mTag,tsnk))
                # if-analyze
            # for tsnk
        # for momentum
        
           
print('\nDone\n')

## Construct Ratio of three- and two-point functions, $R(t_s,t_{\rm ins}) = C_{3pt}(t_s,t_{\rm ins})\; /\; C_{2pt}(t_s)$

In [ ]:

R_bins_Re = {}
R_bins_Im = {}

R_ave_Re = {}
R_ave_Im = {}


for opT in SrcSnkOpTypes:
    for phT in phaseTypes:
        momTags = momTags_Dict[(phT,opT)]
                
        for mTag in momTags:
            mVec = momVector_Dict[mTag]
            zMom = mVec[2]
            mFTag = momFileTag(mVec)
            pmTpl = phMomTpl(phT,zMom)

            srcOpList = srcOpList3pt_Dict[opT][pmTpl] # These are the same in two- and
            snkOpList = srcOpList3pt_Dict[opT][pmTpl] # three-point functions
        
            for isrc, src in enumerate(srcOpList):
                for isnk, snk in enumerate(snkOpList):
                    ckey2 = (opT,phT,mTag,isrc,isnk) # 2pt-tuple

                    for tsnk in TsinkList[(phT,opT)]:
                        tsnkTag = 'tsnk_%d'%(tsnk)
                        Ntins = tsnk
                        
                        dTpl = (phT,opT,mTag,tsnk)
                        Nbins = Nbins_3pt_Dict[dTpl]
                        
                        adTplM = (phT,opT,mTag,tsnk)
                        if analyzeDataSetM[adTplM]:
                            for irowSrc in range(1,nRow+1):
                                for irowSnk in range(1,nRow+1):                        
                                    for gMat in gMatList:
                                        insOpName = gMatMap[phT][gMat][0]
                                        insOpRow  = gMatMap[phT][gMat][1]

                                        for z in dispLen:
                                            ckey3 = (opT,phT,mTag,tsnk,isrc,isnk,gMat,z,irowSrc,irowSnk) # 3pt-tuple

                                            R_bins_Re[ckey3] = np.zeros((Nbins,Ntins))
                                            R_bins_Im[ckey3] = np.zeros((Nbins,Ntins))
                                            for tins in range(Ntins):
                                                R_bins_Re[ckey3][:,tins] = c3pt_bins_Re[ckey3][:,tins].real / c2pt_bins[ckey2][:,tsnk]
                                                R_bins_Im[ckey3][:,tins] = c3pt_bins_Im[ckey3][:,tins].imag / c2pt_bins[ckey2][:,tsnk]

                                            R_ave_Re[ckey3] = jackknife_ave(R_bins_Re[ckey3],Nbins,tL=Ntins)
                                            R_ave_Im[ckey3] = jackknife_ave(R_bins_Im[ckey3],Nbins,tL=Ntins)


print('\nDone\n')

### Plot ratio $R(t_s,t_{\rm ins})$

In [ ]:
figR, axR = plt.subplots(figsize=[15,9.4])

opTR = 'single'
phTR = 'unphased'
isrcR = 0
isnkR = 0
irowSrcR = 1
irowSnkR = 1

gMatR = 'gt'

mTagR = 'pz.+1'

dL = 3

i = 0
for tsink in TsinkList[(phTR,opTR)]:
    
    xRng = np.arange(tsink) - tsink/2
    
    adTplM = (phTR,opTR,mTagR,tsink)
    if analyzeDataSetM[adTplM]:
        ckey = (opTR,phTR,mTagR,tsink,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)

        axR.errorbar(xRng[1:]+osDctF[i],  R_ave_Re[ckey][0][1:],
                     yerr = R_ave_Re[ckey][1][1:],
                     color = cDct[i], marker = mDct[i], ls = 'none',
                     label=r'$t_s=%d$'%(tsink),
                     ms = 8.0, elinewidth=elw, capsize=ecs,fillstyle='full')
        i+=1

        
axR.set_xlabel(r'$[(t_{\rm ins} -t_s)/2]/a$',fontsize=18)
axR.set_ylabel(r'$R(\vec{p};z_3;t_{\rm ins},t_s;%s)$'%(gMatLatex[gMatR]),fontsize=18)

axR.xaxis.set_ticks(np.arange(-7, 7, 1))
#axR.set_ylim(0.5,1.2)
#axR.yaxis.set_ticks(np.arange(0.45, 0.86, 0.05))

axR.legend(loc='best', ncol=2, numpoints=1, fontsize=18,
           labelspacing=0.2,borderpad=0.2,columnspacing=0.2,
           markerscale=0.8)